# Formality Classifier
This is going to be used to classify whether a sentence should be included in the informal or formal corpus. This will work by selecting the probability of the sentence belonging to the corpus, and if the score exceeds a threshold it will be included. 

In [1]:
import numpy as np
import seaborn as sns
import tensorflow as tf

import re 
import os
import pickle
import json
from datetime import datetime

from nltk.translate.bleu_score import sentence_bleu

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

### Static Variables

In [2]:
BATCH_SIZE = 64
EMBEDDING_DIM = 200

## Load Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# BASE_PATH = '../../Data'  # on local is path to directory
BASE_PATH = '/content/drive/MyDrive/Data/Data'

FORMAL_PATH_TRAIN = '{}/Supervised Data/Family_Relationships/S_Formal_FR_train.txt'.format(BASE_PATH)
INFORMAL_PATH_TRAIN = '{}/Supervised Data/Family_Relationships/S_Informal_FR_train.txt'.format(BASE_PATH)

FORMAL_PATH_HOLDOUT = '{}/Supervised Data/Family_Relationships/S_Formal_FR_ValTest.txt'.format(BASE_PATH)
INFORMAL_PATH_HOLDOUT = '{}/Supervised Data/Family_Relationships/S_Informal_FR_ValTest.txt'.format(BASE_PATH)

EMBEDDING_PATH = '{}/glove.6B.200d.txt'.format(BASE_PATH)

In [5]:
formal = open(FORMAL_PATH_TRAIN).read()
informal = open(INFORMAL_PATH_TRAIN).read()

formal_holdout = open(FORMAL_PATH_HOLDOUT).read()
informal_holdout = open(INFORMAL_PATH_HOLDOUT).read()

### Preprocess data

In [6]:
def process_sequence(seq):
    """This inserts a space in between the last word and a period"""
    s = re.sub('([.,!?()])', r' \1 ', seq)
    s = re.sub('\s{2,}', ' ', s)
    
    return '<start> ' + s + ' <end>'

In [7]:
f_corpus = [process_sequence(seq) for seq in formal.split('\n')]
if_corpus = [process_sequence(seq) for seq in informal.split('\n')]

f_holdout = [process_sequence(seq) for seq in formal_holdout.split('\n')]
if_holdout = [process_sequence(seq) for seq in informal_holdout.split('\n')]

In [8]:
input_corpus = f_corpus.copy()
input_corpus.extend(if_corpus)

input_labels = [True for _ in range(len(f_corpus))]
input_labels.extend([False for _ in range(len(if_corpus))])

In [9]:
holdout_corpus = f_holdout.copy()
holdout_corpus.extend(if_holdout)

holdout_labels = [True for _ in range(len(f_holdout))]
holdout_labels.extend([False for _ in range(len(if_holdout))])

In [10]:
i = 26000
print(input_corpus[i])
print(input_labels[i])

<start> obviously this woman's priorities are all messed up and she needs total help .  <end>
False


### Tokenize

In [11]:
def tokenize(corpus, tokenizer=None, maxlen=None):
    """ Tokenize data and pad sequences """
    if not tokenizer: 
        tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n', 
                              oov_token='<OOV>')
        tokenizer.fit_on_texts(corpus)
    
    seqs = tokenizer.texts_to_sequences(corpus)
    padded_seqs = pad_sequences(seqs, padding='post', maxlen=maxlen)

    return padded_seqs, tokenizer

In [12]:
train_set, tokenizer = tokenize(input_corpus)
test_set, _ = tokenize(holdout_corpus, tokenizer)

In [13]:
train_set[0]

array([  2,   6, 349,   5, 124,   9,  74,  63,  35,   3,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0], dtype=int32)

### Setup TF dataset

In [14]:
buffer_size = len(train_set)
steps_per_epoch = len(train_set) // BATCH_SIZE
vocab_size = len(tokenizer.word_index) + 1

train = tf.data.Dataset.from_tensor_slices((train_set, input_labels)).shuffle(buffer_size)
train = train.batch(BATCH_SIZE, drop_remainder=True)

test = tf.data.Dataset.from_tensor_slices((test_set, holdout_labels))
test = test.batch(BATCH_SIZE)

In [15]:
example_input_batch, example_target_batch = next(iter(train))

### Load Embedding Weights

In [16]:
def embedding_matrix(tokenizer, vocab_size, embedding_dim):
    embeddings_index = {}
    with open(EMBEDDING_PATH) as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs

    embeddings_matrix = np.zeros((vocab_size, embedding_dim))
    for word, i in tokenizer.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embeddings_matrix[i] = embedding_vector

    return embeddings_matrix

In [17]:
E = embedding_matrix(tokenizer, vocab_size, EMBEDDING_DIM)

## Declare Model

In [56]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, EMBEDDING_DIM, weights=[E], mask_zero=True), 
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(1024, return_sequences=True),
    tf.keras.layers.LSTM(1024),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [57]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [58]:
history = model.fit(train, epochs=10)

Epoch 1/10
781/781 [==============================] - 46s 58ms/step - loss: 0.5068 - accuracy: 0.7376
Epoch 2/10
781/781 [==============================] - 46s 59ms/step - loss: 0.4267 - accuracy: 0.7925
Epoch 3/10
781/781 [==============================] - 46s 59ms/step - loss: 0.3904 - accuracy: 0.8114
Epoch 4/10
781/781 [==============================] - 46s 58ms/step - loss: 0.3626 - accuracy: 0.8252
Epoch 5/10
781/781 [==============================] - 45s 58ms/step - loss: 0.3406 - accuracy: 0.8358
Epoch 6/10
781/781 [==============================] - 46s 59ms/step - loss: 0.3173 - accuracy: 0.8456
Epoch 7/10
781/781 [==============================] - 46s 58ms/step - loss: 0.2963 - accuracy: 0.8544
Epoch 8/10
781/781 [==============================] - 46s 58ms/step - loss: 0.2765 - accuracy: 0.8652
Epoch 9/10
781/781 [==============================] - 45s 58ms/step - loss: 0.2567 - accuracy: 0.8730
Epoch 10/10
781/781 [==============================] - 45s 58ms/step - loss: 0.237

In [59]:
model.evaluate(test)

62/62 [==============================] - 1s 15ms/step - loss: 0.5985 - accuracy: 0.7896


[0.5984866619110107, 0.7896341681480408]

Going to find the following data set 
$$ T_{avg} = \{(s_i, s_i^\prime)|P_+(s_i^\prime) - P_+(s_i) > \sigma \}$$

In [44]:
model.predict(test)

array([[0.49983445, 0.5001656 ],
       [0.4999894 , 0.5000106 ],
       [0.5001057 , 0.49989432],
       ...,
       [0.500086  , 0.49991402],
       [0.5000835 , 0.49991652],
       [0.50109226, 0.49890772]], dtype=float32)